In [0]:
# Databricks Notebook: Gold Summary for Dashboard Boxes
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# ========== 설정 ==========
CATALOG  = "demo_catalog"
SCHEMA   = "demo_schema"
GOLD_15  = f"{CATALOG}.{SCHEMA}.gold_prices_15m"
GOLD_4H  = f"{CATALOG}.{SCHEMA}.gold_prices_4h"
FNG_SLV  = f"{CATALOG}.{SCHEMA}.silver_fear_greed"
SUMMARY  = f"{CATALOG}.{SCHEMA}.gold_dashboard_summary"

# ========== 요약 테이블 ==========
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {SUMMARY} (
  symbol          STRING,
  latest_price    DOUBLE,
  change_24h_pct  DOUBLE,
  cross_signal_4h STRING,
  fgi_value       INT,
  fgi_class       STRING,
  updated_at      TIMESTAMP
) USING DELTA
""")

# 최신 15m (심볼별 가장 최근)
g15 = spark.table(GOLD_15).select("symbol","open_time","close","pct_change_24h")
w15 = Window.partitionBy("symbol").orderBy(F.col("open_time").desc())
latest_15 = (
  g15.withColumn("rn", F.row_number().over(w15))
     .where("rn = 1")
     .select(
        F.col("symbol"),
        F.col("close").alias("latest_price"),
        F.col("pct_change_24h").alias("change_24h_pct")
     )
)

# 최신 4h cross
g4 = spark.table(GOLD_4H).select("symbol","bucket_start","cross_signal")
w4 = Window.partitionBy("symbol").orderBy(F.col("bucket_start").desc())
latest_4h = (
  g4.withColumn("rn", F.row_number().over(w4))
     .where("rn = 1")
     .select(F.col("symbol").alias("sym4"), F.col("cross_signal").alias("cross_signal_4h"))
)

# 최신 F&G (전역 지표이므로 전체 중 1행만 최신)
fng = spark.table(FNG_SLV).select("event_time","index_value","value_classification")
fng_latest = (
  fng.orderBy(F.col("event_time").desc()).limit(1)
     .withColumnRenamed("index_value","fgi_value")
     .withColumnRenamed("value_classification","fgi_class")
     .withColumn("dummy", F.lit(1))
)

# 심볼 목록에 F&G 최신(1행)을 cross join 해서 모두에 부여
symbols_df = latest_15.select("symbol").distinct().withColumn("dummy", F.lit(1))
summary_df = (
  latest_15
    .join(latest_4h, latest_15.symbol == latest_4h.sym4, "left")
    .drop("sym4")
    .join(symbols_df, "symbol", "left")
    .join(fng_latest, "dummy", "left")
    .drop("dummy")
    .withColumn("updated_at", F.current_timestamp())
    .select("symbol","latest_price","change_24h_pct","cross_signal_4h","fgi_value","fgi_class","updated_at")
)

# 간단히 덮어쓰기(요약이라 전체 새로 고침이 편함)
summary_df.write.mode("overwrite").saveAsTable(SUMMARY)

print(f"[SUMMARY] refreshed: {SUMMARY}")
